In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import joblib
from IPython.display import display

In [2]:
# Define the custom R^2 metric for the Keras model
def r2_keras(output_index):
    def r2(y_true, y_pred):
        SS_res = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - y_pred[:, output_index]))
        SS_tot = tf.keras.backend.sum(tf.keras.backend.square(y_true[:, output_index] - tf.keras.backend.mean(y_true[:, output_index])))
        r2 = 1 - SS_res / (SS_tot + tf.keras.backend.epsilon())
        return r2
    r2.__name__ = f"r2_{output_index}"
    return r2

In [3]:
# Load the Excel file
file_path = 'cleaned manually cell 1 and feed.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1', header=[1, 2])

# Display the first few rows of the data
display(df.head())

# Define the inputs and output for the model
feature_columns = {
    'Net Volume': 48, 
    'Pulp Area': 28, 
    'Froth surface area': 20, 
    'Froth thickness': 250, 
    'Air Flow rate': 18, 
    'R_inf Ccp': 30, 
    'R_inf Gn': 85, 
    'R_inf Po': 20, 
    'R_inf Sp': 17, 
    'k_max Ccp': 2.2, 
    'k_max Gn': 2.2, 
    'k_maxPo': 2.2, 
    'k_max Sp': 2.2, 
    'Entrainement Savassi parameters': ('Cellule 1', 'Entrainement Savassi parameter_Cellule1'), 
    'Total Solids Flow_Feed': ('Raw feed', 'Total Solids Flow_input'), 
    'Total Liquid Flow_Feed': ('Raw feed', 'Total Liquid Flow_input'), 
    'Pulp Volumetric Flow_Feed': ('Pb Conditioner', 'Pulp Volumetric Flow'), 
    'Solids SG_Feed': ('Pb Conditioner', 'Solids SG'), 
    'Pulp SG_Feed': ('Pb Conditioner', 'Pulp SG'), 
    'Solids Fraction_Feed': ('Pb Conditioner', 'Solids Fraction'), 
    'Cu_Feed': ('Pb Conditioner', 'Cu'), 
    'Fe_Feed': ('Pb Conditioner', 'Fe'), 
    'Pb_Feed': ('Pb Conditioner', 'Pb'), 
    'Zn_Feed': ('Pb Conditioner', 'Zn')
}


# Extract the feed data
feed_data = pd.DataFrame()
for feature, column in feature_columns.items():
    if isinstance(column, tuple):
        head, subhead = column
        if (head, subhead) in df.columns:
            feed_data[feature] = df[(head, subhead)]
        else:
            print(f"Column {head}, {subhead} not found in DataFrame")
    else:
        feed_data[feature] = column

# Add static values
static_values = {
    'Net Volume': 48, 
    'Pulp Area': 28, 
    'Froth surface area': 20, 
    'Froth thickness': 250, 
    'Air Flow rate': 18, 
    'R_inf Ccp': 30, 
    'R_inf Gn': 85, 
    'R_inf Po': 20, 
    'R_inf Sp': 17, 
    'k_max Ccp': 2.2, 
    'k_max Gn': 2.2, 
    'k_maxPo': 2.2, 
    'k_max Sp': 2.2
}

for feature, value in static_values.items():
    feed_data[feature] = value

# Convert the feed data to the correct dtype
feed_data = feed_data.astype(float)

Unit Name  -                Raw feed                                 \
  Variable Name  - Total Solids Flow_input Total Liquid Flow_input Gn Sph   
0         Run 1  5                      50                     130  1   1   
1         Run 2  5                      60                     135  2   2   
2         Run 3  5                      70                     140  3   3   

                                        Cellule 1    Pb Conditioner  ...  \
  Chlc Po Entrainement Savassi parameter_Cellule1 Total Solids Flow  ...   
0    1  6                                      40                50  ...   
1    2  7                                       6                60  ...   
2    3  8                                       7                70  ...   

    Cellule 1                                                           \
  Liquid SG.1 Pulp SG.1 % Solids.1 Solids Fraction.1 Solids Recovery.1   
0    0.997009  1.232710  27.863578         10.809930        101.882355   
1    0.997009  1.264419  30.717140         12.134609         97.842991   
2    0.997009  1.296243  33.333333         13.324543         96.683507   

                                                              
  Liquid Recovery.1      Cu.1       Fe.1      Pb.1      Zn.1  
0        101.447965  3.499262  12.778598  0.180749  1.014787  
1         98.082659  3.404092  13.134785  0.321347  1.376876  
2         96.683507  3.405709  13.817888  0.487810  2.058938  

[3 rows x 57 columns]

In [4]:
display(feed_data)
feed_data.to_csv('modelInput.csv', index=False)
# Load scalers
scalerX = joblib.load('scalerX_24.joblib')
scalerY = joblib.load('scalerY_23.joblib')

# Scale the input data
scaled_feed_data = scalerX.transform(feed_data)

# Load the saved model
custom_objects = {f'r2_{i}': r2_keras(i) for i in range(23)}
model = load_model('model7_23.h5', custom_objects=custom_objects)

,Net Volume,Pulp Area,Froth surface area,Froth thickness,Air Flow rate,R_inf Ccp,R_inf Gn,R_inf Po,R_inf Sp,k_max Ccp,...,Total Solids Flow_Feed,Total Liquid Flow_Feed,Pulp Volumetric Flow_Feed,Solids SG_Feed,Pulp SG_Feed,Solids Fraction_Feed,Cu_Feed,Fe_Feed,Pb_Feed,Zn_Feed
0,48.0,28.0,20.0,250.0,18.0,30.0,85.0,20.0,17.0,2.2,...,50.0,130.0,146.119522,3.178736,1.231868,10.764833,3.463,12.42924,0.866,0.671
1,48.0,28.0,20.0,250.0,18.0,30.0,85.0,20.0,17.0,2.2,...,60.0,135.0,153.941303,3.236891,1.266717,12.041150,3.463,13.03974,1.732,1.342
2,48.0,28.0,20.0,250.0,18.0,30.0,85.0,20.0,17.0,2.2,...,70.0,140.0,161.650042,3.297214,1.299103,13.133336,3.463,13.65024,2.598,2.013


C:\Users\czeroo\Desktop\Managem App\SimulationApp\internship\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:

# Predict the outputs
predictions = model.predict(scaled_feed_data)

# Invert the scaling to get the original values of the predictions
predictions = scalerY.inverse_transform(predictions)

# Extract relevant columns from predictions
concentrate_pred = predictions[:, [3, 4, 5, 6, 7, 8, 19, 20, 21, 22]]
tailing_pred = predictions[:, 9:19]

1/1 [==============================] - ETA: 0s

1/1 [==============================] - 0s 331ms/step


In [6]:
# Define the keys for the outputs
concentrate_keys = [
    'totalSolidFlow', 'totalLiquidFlow', 'pulpVolumetricFlow', 
    'solidsSG', 'pulpSG', 'solidsFraction', 
    'cuPercentage', 'fePercentage', 'znPercentage', 'pbPercentage'
]

tailing_keys = [
    'totalSolidFlow', 'totalLiquidFlow', 'pulpVolumetricFlow', 
    'solidsSG', 'pulpSG', 'solidsFraction', 
    'cuPercentage', 'fePercentage', 'znPercentage', 'pbPercentage'
]

# Create dataframes for the predictions
concentrate_pred_df = pd.DataFrame(concentrate_pred, columns=concentrate_keys)
tailing_pred_df = pd.DataFrame(tailing_pred, columns=tailing_keys)

# Extract the actual outputs from the sheet
actual_concentrate_columns = [
    'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate', 
    'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 
    'Pulp SG_Concentrate', 'Solids Fraction_Concentrate', 
    'Cu_Concentrate', 'Fe_Concentrate', 'Zn_Concentrate', 'Pb_Concentrate'
]

actual_tailing_columns = [
    'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings', 
    'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 
    'Pulp SG_Tailings', 'Solids Fraction_Tailings', 
    'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails'
]

actual_concentrate = df.iloc[2:, [47, 48, 49, 50, 51, 52, 53, 54, 55, 56]]
actual_concentrate.columns = actual_concentrate_columns

actual_tailing = df.iloc[2:, [28, 29, 30, 31, 32, 33, 34, 35, 36, 37]]
actual_tailing.columns = actual_tailing_columns

In [7]:
# Display the actual and predicted values for comparison
display(actual_concentrate.head())
display(concentrate_pred_df.head())

display(actual_tailing.head())
display(tailing_pred_df.head())

,Total Solids Flow_Concentrate,Total Liquid Flow_Concentrate,Pulp Volumetric Flow_Concentrate,Solids SG_Concentrate,Pulp SG_Concentrate,Solids Fraction_Concentrate,Cu_Concentrate,Fe_Concentrate,Zn_Concentrate,Pb_Concentrate
2,0.997009,1.296243,33.333333,13.324543,96.683507,96.683507,3.405709,13.817888,0.48781,2.058938


,totalSolidFlow,totalLiquidFlow,pulpVolumetricFlow,solidsSG,pulpSG,solidsFraction,cuPercentage,fePercentage,znPercentage,pbPercentage
0,238.972443,213.406525,380.862610,6.047151,1.413165,11.517680,8.456583,23.264027,58.663250,3.555535
1,217.899582,196.904099,341.174255,5.979405,1.411312,11.478133,8.100338,23.260843,57.441551,4.324800
2,212.144089,194.756470,335.863586,5.948892,1.410490,11.470573,8.029708,23.329969,56.753571,4.395338


,Total Solids Flow_Tailings,Total Liquid Flow_Tailings,Pulp Volumetric Flow_Tailings,Solids SG_Tailings,Pulp SG_Tailings,Solids Fraction_Tailings,Cu_Tails,Fe_Tails,Pb_Tails,Zn_Tails
2,48.566148,36.562805,3.959825,0.997009,1.328294,33.333333,11.181419,23.126737,23.126737,7.066937


,totalSolidFlow,totalLiquidFlow,pulpVolumetricFlow,solidsSG,pulpSG,solidsFraction,cuPercentage,fePercentage,znPercentage,pbPercentage
0,25.436443,136.196121,127.687782,4.005911,1.948199,36.666039,7.366023,25.257755,10.739371,7.371535
1,21.164093,126.069038,117.754555,4.015993,1.960654,37.010975,7.166850,25.915897,10.130059,8.171991
2,21.888996,121.426537,112.730820,4.022650,2.005423,37.948643,7.169741,26.127583,10.052086,8.228024
